In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..')

In [3]:
import osmnx as ox
import fiona
import geopandas as gpd
from lonboard import Map, PolygonLayer, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
import numpy as np

In [4]:
from main import prepare_data_for_place, OUTPUT_COLUMNS

In [5]:
def is_connected(g):
    import networkx as nx
    return nx.is_connected(g.to_undirected())

## Prepare out dir

In [28]:
import os
import shutil

output_path = "../data/out/notebook/"

# Delete the directory if it exists
if os.path.exists(output_path):
    shutil.rmtree(output_path)

# Recreate the directory
os.makedirs(output_path, exist_ok=True)

## Load bike network

In [8]:
nodes, edges = prepare_data_for_place("Somerville, MA, USA")

> Getting bike network for Somerville, MA, USA
> Processing network for Somerville, MA, USA
> MODEL 1: Preparing speed data for Somerville, MA, USA
> MODEL 2: Preparing separation level data for Somerville, MA, USA
> MODEL 3: Preparing street category data for Somerville, MA, USA
> MODEL 4: Preparing lanes data for Somerville, MA, USA
> MODEL: Preparing composite score for Somerville, MA, USA


In [9]:
edges = edges[OUTPUT_COLUMNS]

In [10]:
G = ox.graph_from_gdfs(nodes, edges)

In [11]:
edges.sample(3)

,,,name,maxspeed_0,maxspeed_int,maxspeed_int_score,separation_level,separation_level_score,street_0,street_classification,street_classification_score,composite_score,width_float,width_half,geometry
u,v,key,,,,,,,,,,,,,
780,779,0,Edgar Terrace,25 mph,25.0,1.0,none,4.0,residential,residential,2,3.0,12.2,6.1,"LINESTRING (-71.09703 42.39685, -71.097 42.396..."
1423,1422,0,SomerNOVA Alley,NaN,NaN,NaN,none,4.0,service,residential,2,3.5,10.0,5.0,"LINESTRING (-71.1027 42.38127, -71.10228 42.38..."
236,235,0,Union Square,20 mph,20.0,0.0,separate,0.0,primary,medium-capacity,3,0.6,19.8,9.9,"LINESTRING (-71.09515 42.37946, -71.09521 42.3..."


In [12]:
is_connected(G)

True

## Load Schools

In [13]:
school_gdb_path = "../data/raw/SafeRoutesGISLayers.gdb.zip"

In [14]:
layers = fiona.listlayers(school_gdb_path)
layers

['SafetyZoneStreets', 'PublicSchools', 'Sidewalks', 'Signalized_Intersections']

In [15]:
# read school data
schools_gdf = gpd.read_file(school_gdb_path, layer='PublicSchools')

# save schools polygons
schools_gdf.to_file("../data/out/gpkg/schools_poly.gpkg", driver="GPKG")

# make geom col into centroids
schools_gdf['geometry'] = schools_gdf.centroid

# save schools polygons
schools_gdf.to_file("../data/out/gpkg/schools_centroid.gpkg", driver="GPKG")

In [16]:
schools_gdf.head(3)

,Name,GlobalID,Shape_Length,Shape_Area,geometry
0,West Somerville Neighborhood School,{423648E4-357B-4C51-8323-18DE5B5EF869},857.126130,20546.222891,POINT (757029.484 2973287.291)
1,Brown School,{32ED129B-38AE-4E8F-A71B-A18126973D75},511.378543,10156.639765,POINT (760400.444 2970061.762)
2,Healey School,{374CFA80-E38D-4411-AB46-7868E8DA8468},900.132189,38897.272280,POINT (765459.28 2970148.61)


## Load census blocks

In [27]:
# read census blocks
census_blocks = gpd.read_file("../data/raw/Census_2020_Blocks.zip")

# filter by TOWN attribute
somerville_census_blocks = census_blocks[census_blocks['TOWN'] == "Somerville"].copy()

# reset index
somerville_census_blocks = somerville_census_blocks.reset_index(drop=True)

# save polygon version
somerville_census_blocks.to_file("../data/out/gpkg/somerville_blocks_poly.gpkg", driver="GPKG")

# convert geometry to centroid
somerville_census_blocks['geometry'] = somerville_census_blocks.centroid

# save centroid version
somerville_census_blocks.to_file("../data/out/gpkg/somerville_blocks_centroid.gpkg", driver="GPKG")


In [18]:
somerville_census_blocks.head(3)

,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,ALAND20,AWATER20,...,AREA_SQFT,AREA_ACRES,TOWN,TOWN_ID,BLKGRP20,TRACT20,COUSUBFP,SHAPEAREA,SHAPELEN,geometry
0,74801,25,017,352101,3002,250173521013002,Block 3002,G5040,11959,0,...,128715.40,2.95,CAMBRIDGE,49,250173521013,25017352101,11000,11958.100112,637.708886,POINT (235036.811 902558.718)
1,67852,25,017,351500,4010,250173515004010,Block 4010,G5040,2460,0,...,26480.05,0.61,SOMERVILLE,274,250173515004,25017351500,62535,2460.086623,244.934565,POINT (235065.002 902556.345)
2,78290,25,017,352200,1001,250173522001001,Block 1001,G5040,11470,0,...,123451.71,2.83,CAMBRIDGE,49,250173522001,25017352200,11000,11469.085483,445.949763,POINT (233986.639 902622.082)


In [19]:
somerville_census_blocks.to_file("../data/out/blockgroups.gpkg", driver="GPKG")

## Make sure everything has same crs

- EPSG:26986 =  NAD83 / Massachusetts Mainland Meters
- EPSG:4326 = WGS 84 / web

In [20]:
def crs_first_line(gdf):
    return str(gdf.crs).splitlines()[0]

In [21]:
print("somerville_census_blocks:", crs_first_line(somerville_census_blocks))
print("schools_gdf             :", crs_first_line(schools_gdf))
print("edges                   :", crs_first_line(edges))
print("nodes                   :", crs_first_line(nodes))

somerville_census_blocks: EPSG:26986
schools_gdf             : EPSG:6492
edges                   : EPSG:4326
nodes                   : EPSG:4326


In [22]:
# use this one
use_crs = edges.crs
# use_crs = somerville_census_blocks.crs
# use_crs = projected_crs

# make them match
somerville_census_blocks = somerville_census_blocks.to_crs(use_crs)
schools_gdf = schools_gdf.to_crs(use_crs)
edges = edges.to_crs(use_crs)

In [23]:
print("somerville_census_blocks:", crs_first_line(somerville_census_blocks))
print("schools_gdf             :", crs_first_line(schools_gdf))
print("edges                   :", crs_first_line(edges))
print("nodes                   :", crs_first_line(nodes))

somerville_census_blocks: EPSG:4326
schools_gdf             : EPSG:4326
edges                   : EPSG:4326
nodes                   : EPSG:4326


## Routing

In [24]:
G = ox.graph_from_gdfs(nodes, edges)

orig_point = somerville_census_blocks.loc[0, 'geometry']
dest_point = schools_gdf.loc[0, 'geometry']

orig = ox.nearest_nodes(G, X=orig_point.x, Y=orig_point.y)
dest = ox.nearest_nodes(G, X=dest_point.x, Y=dest_point.y)

route = ox.routing.shortest_path(G, orig, dest, weight="composite_score")

### Save route

In [25]:
from shapely.geometry import Point, LineString

# --- 1. Create geometry for start, end, and route ---
start_geom = Point(orig_point.x, orig_point.y)
end_geom = Point(dest_point.x, dest_point.y)

# Convert the route node IDs to coordinates
route_coords = [(G.nodes[n]['x'], G.nodes[n]['y']) for n in route]
route_geom = LineString(route_coords)

# --- 2. Create GeoDataFrames ---
gdf = gpd.GeoDataFrame([
    {'type': 'start', 'geometry': start_geom},
    {'type': 'end', 'geometry': end_geom},
    {'type': 'route', 'geometry': route_geom}
], crs="EPSG:4326")

# --- 3. Save to GeoPackage ---
gdf.to_file("../data/out/route.gpkg", driver="GPKG")